In [82]:
using DataFrames, DataFramesMeta, Gadfly, Compose, XGBoost, Lazy

INFO: Recompiling stale cache file /Users/kburke/.julia/lib/v0.4/Lazy.ji for module Lazy.


In [335]:
t = readtable("titanic/test.csv")
t[1:4, :]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NA,Q
2,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0,NA,S
3,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NA,Q
4,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NA,S


In [469]:
function fixData(df, includeCategory = false)
    meanAge = mean(df[!isna(df[:Age]),:Age])
    df[isna(df[:Age]),:Age] = meanAge
    
    meanFare = mean(df[!isna(df[:Fare]),:Fare])
    df[isna(df[:Fare]),:Fare] = meanFare
    
    df[isna(df[:Embarked]),:Embarked] = "Q"
    
    categoryCols = [(:Embarked, :EmbarkedLabel),(:Sex, :SexLabel)]
    
    cols = [:Pclass, :Age, :SibSp, :Parch, :Fare, :EmbarkedLabel, :SexLabel]
    if (includeCategory)
        push!(cols, :Survived)
    end
    
    pool!(df)
    
    [df[c[2]] = convert(Array{Int64,1},[getpoolidx(df[c[1]], v) for v in df[c[1]]]) for c in categoryCols]
    pool!(df)
    
    (df, @select(df, cols))
end

fixData (generic function with 2 methods)

In [470]:
(withId, test) = fixData(readtable("titanic/test.csv"))
(_, train) = fixData(readtable("titanic/train.csv"),true)

(891×14 DataFrames.DataFrame
│ Row │ PassengerId │ Survived │ Pclass │
├─────┼─────────────┼──────────┼────────┤
│ 1   │ 1           │ 0        │ 3      │
│ 2   │ 2           │ 1        │ 1      │
│ 3   │ 3           │ 1        │ 3      │
│ 4   │ 4           │ 1        │ 1      │
│ 5   │ 5           │ 0        │ 3      │
│ 6   │ 6           │ 0        │ 3      │
│ 7   │ 7           │ 0        │ 1      │
│ 8   │ 8           │ 0        │ 3      │
│ 9   │ 9           │ 1        │ 3      │
│ 10  │ 10          │ 1        │ 2      │
│ 11  │ 11          │ 1        │ 3      │
⋮
│ 880 │ 880         │ 1        │ 1      │
│ 881 │ 881         │ 1        │ 2      │
│ 882 │ 882         │ 0        │ 3      │
│ 883 │ 883         │ 0        │ 3      │
│ 884 │ 884         │ 0        │ 2      │
│ 885 │ 885         │ 0        │ 3      │
│ 886 │ 886         │ 0        │ 3      │
│ 887 │ 887         │ 0        │ 2      │
│ 888 │ 888         │ 1        │ 1      │
│ 889 │ 889         │ 0        │ 3      │
│ 8

In [312]:
num_pas = size(train)[1]

891

In [313]:
num_survived = size(train[train[:Survived] .== 1, :])[1]

342

In [314]:
proportion_survivors =  num_survived / num_pas

0.3838383838383838

In [316]:
men_only_stats= train[train[:isMale], :]
women_only_stats= train[!train[:isMale], :];
men_survived = size(@where(men_only_stats, :Survived .== 1))[1]/ size(men_only_stats)[1] 

0.18890814558058924

In [317]:
women_survived = size(@where(women_only_stats, :Survived .== 1))[1]/ size(women_only_stats)[1] 

0.7420382165605095

In [491]:
noLabel = deepcopy(train)
delete!(noLabel, :Survived)
print(size(noLabel))
dtrain= DMatrix(convert(Array,noLabel[1:700,:]), label = convert(Array, train[:Survived]))
testSet = noLabel[701:891,:]
dtestSet= DMatrix(convert(Array,testSet))

(891,7)

XGBoost.DMatrix(Ptr{Void} @0x00007ffea724ace0,_setinfo)

In [482]:
numRound = 700
bst = xgboost(dtrain, numRound, objective="binary:logistic", eta=.1,maxdepth=10)

[1]	train-error:0.133558
[2]	train-error:0.136925
[3]	train-error:0.136925
[4]	train-error:0.136925
[5]	train-error:0.136925
[6]	train-error:0.136925
[7]	train-error:0.136925
[8]	train-error:0.134680
[9]	train-error:0.127946
[10]	train-error:0.124579
[11]	train-error:0.120090
[12]	train-error:0.122334
[13]	train-error:0.123457
[14]	train-error:0.124579
[15]	train-error:0.124579
[16]	train-error:0.121212
[17]	train-error:0.120090
[18]	train-error:0.120090
[19]	train-error:0.120090
[20]	train-error:0.121212
[21]	train-error:0.120090
[22]	train-error:0.123457
[23]	train-error:0.120090
[24]	train-error:0.118967
[25]	train-error:0.118967
[26]	train-error:0.120090
[27]	train-error:0.118967
[28]	train-error:0.117845
[29]	train-error:0.117845
[30]	train-error:0.117845
[31]	train-error:0.117845
[32]	train-error:0.116723
[33]	train-error:0.116723
[34]	train-error:0.116723
[35]	train-error:0.116723
[36]	train-error:0.115600
[37]	train-error:0.114478
[38]	train-error:0.114478
[39]	train-error:0.11

XGBoost.Booster(Ptr{Void} @0x00007ffea70c81c0)

In [492]:
dtest = DMatrix(convert(Array,test))
pred = XGBoost.predict(bst, dtest)
trainPred = XGBoost.predict(bst, dtestSet)
print("test-error=", sum((trainPred .> 0.5) .!= testSet[:Survived]) / float(size(testSet)[1]), "\n")

LoadError: LoadError: KeyError: Survived not found
while loading In[492], in expression starting on line 4

In [466]:
nfold=5
param = ["max_depth"=>4, "eta"=>.1, "objective"=>"binary:logistic"]
nfold_cv(dtrain, numRound, nfold,  param=param, metrics=["error"], seed = 0)


Use "Dict(a=>b, ...)" instead.
[1]	cv-test-error:0.184081+0.028613	cv-train-error:0.162741+0.010356
[2]	cv-test-error:0.189693+0.020602	cv-train-error:0.159654+0.004556
[3]	cv-test-error:0.185192+0.025185	cv-train-error:0.161057+0.008146
[4]	cv-test-error:0.187446+0.025133	cv-train-error:0.161337+0.006861
[5]	cv-test-error:0.187465+0.024634	cv-train-error:0.158532+0.004960
[6]	cv-test-error:0.185217+0.025372	cv-train-error:0.157691+0.006171
[7]	cv-test-error:0.187477+0.028281	cv-train-error:0.156568+0.005105
[8]	cv-test-error:0.186347+0.026454	cv-train-error:0.156567+0.004580
[9]	cv-test-error:0.189705+0.022538	cv-train-error:0.150113+0.005168
[10]	cv-test-error:0.184087+0.025133	cv-train-error:0.152360+0.007734
[11]	cv-test-error:0.181847+0.027320	cv-train-error:0.152360+0.006552
[12]	cv-test-error:0.184075+0.026567	cv-train-error:0.147306+0.003692
[13]	cv-test-error:0.184075+0.025351	cv-train-error:0.148709+0.002625
[14]	cv-test-error:0.180710+0.028586	cv-train-error:0.147026+0.0042

tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=4
tree pruning end, 1 roots, 22 extra nodes, 0 pruned nodes, max_depth=4
tree pruning end, 1 roots, 22 extra nodes, 0 pruned nodes, max_depth=4
tree pruning end, 1 roots, 16 extra nodes, 0 pruned nodes, max_depth=4
tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=4
tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=4
tree pruning end, 1 roots, 20 extra nodes, 0 pruned nodes, max_depth=4
tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=4
tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=4
tree pruning end, 1 roots, 20 extra nodes, 0 pruned nodes, max_depth=4
tree pruning end, 1 roots, 22 extra nodes, 0 pruned nodes, max_depth=4
tree pruning end, 1 roots, 20 extra nodes, 0 pruned nodes, max_depth=4
tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=4
tree pruning end, 1 roots, 20 extra nodes, 0 pruned nodes, max_depth=4
tree p

[43]	cv-test-error:0.177371+0.030650	cv-train-error:0.133279+0.008030
[44]	cv-test-error:0.177371+0.030650	cv-train-error:0.133279+0.008734
[45]	cv-test-error:0.177371+0.030650	cv-train-error:0.131033+0.007976
[46]	cv-test-error:0.178495+0.030606	cv-train-error:0.129631+0.011262
[47]	cv-test-error:0.177371+0.030650	cv-train-error:0.128508+0.010724
[48]	cv-test-error:0.176247+0.029057	cv-train-error:0.128508+0.010296
[49]	cv-test-error:0.177371+0.029064	cv-train-error:0.127385+0.009895
[50]	cv-test-error:0.178488+0.027307	cv-train-error:0.127385+0.009994
[51]	cv-test-error:0.176241+0.026168	cv-train-error:0.125983+0.009173
[52]	cv-test-error:0.176241+0.026168	cv-train-error:0.124300+0.009456
[53]	cv-test-error:0.172883+0.028734	cv-train-error:0.123178+0.011648
[54]	cv-test-error:0.170636+0.027079	cv-train-error:0.122056+0.010479
[55]	cv-test-error:0.172883+0.030596	cv-train-error:0.120653+0.011298


 max_depth=4
tree pruning end, 1 roots, 24 extra nodes, 0 pruned nodes, max_depth=4
tree pruning end, 1 roots, 24 extra nodes, 0 pruned nodes, max_depth=4
tree pruning end, 1 roots, 16 extra nodes, 0 pruned nodes, max_depth=4
tree pruning end, 1 roots, 22 extra nodes, 0 pruned nodes, max_depth=4
tree pruning end, 1 roots, 20 extra nodes, 0 pruned nodes, max_depth=4
tree pruning end, 1 roots, 16 extra nodes, 0 pruned nodes, max_depth=4
tree pruning end, 1 roots, 26 extra nodes, 0 pruned nodes, max_depth=4
tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=4
tree pruning end, 1 roots, 16 extra nodes, 0 pruned nodes, max_depth=4
tree pruning end, 1 roots, 22 extra nodes, 0 pruned nodes, max_depth=4
tree pruning end, 1 roots, 8 extra nodes, 0 pruned nodes, max_depth=4
tree pruning end, 1 roots, 20 extra nodes, 0 pruned nodes, max_depth=4
tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=4
tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_de

[56]	cv-test-error:0.172883+0.028734	cv-train-error:0.118969+0.009516
[57]	cv-test-error:0.170629+0.023623	cv-train-error:0.117847+0.011880
[58]	cv-test-error:0.172877+0.027007	cv-train-error:0.118127+0.011560
[59]	cv-test-error:0.174000+0.028878	cv-train-error:0.117567+0.011483
[60]	cv-test-error:0.172877+0.028151	cv-train-error:0.117286+0.011746
[61]	cv-test-error:0.171753+0.026167	cv-train-error:0.117006+0.012037
[62]	cv-test-error:0.172877+0.026115	cv-train-error:0.115323+0.013822
[63]	cv-test-error:0.172877+0.026115	cv-train-error:0.116164+0.011825
[64]	cv-test-error:0.174000+0.025388	cv-train-error:0.115042+0.013448
[65]	cv-test-error:0.169506+0.022519	cv-train-error:0.115603+0.011413
[66]	cv-test-error:0.170629+0.024282	cv-train-error:0.114481+0.012951
[67]	cv-test-error:0.170629+0.022599	cv-train-error:0.113359+0.012904
[68]	cv-test-error:0.171753+0.021899	cv-train-error:0.113078+0.013398
[69]	cv-test-error:0.174000+0.024438	cv-train-error:0.113639+0.012579
[70]	cv-test-error:0

In [1]:
outdata = DataFrame(PassengerId = withId[:PassengerId],Survived = int(round(pred)))
writetable("titanic/pred.csv", outdata)
outdata

LoadError: LoadError: UndefVarError: test not defined
while loading In[1], in expression starting on line 1

In [511]:
importance(bst, convert(Array{AbstractString},[string(s) for s in names(train)]))

LoadError: LoadError: MethodError: `importance` has no method matching importance(::XGBoost.Booster, ::Array{AbstractString,1})
Closest candidates are:
  importance(::XGBoost.Booster)
  importance(::XGBoost.Booster, !Matched::Array{ASCIIString,1})
while loading In[511], in expression starting on line 1

In [ ]:
using RDatasets
iris = dataset("datasets", "iris")
scmIris = scatterMatrix(@select(train, :Survived, :Pclass,:Age, :SibSp, :Parch, :Fare), "Survived", false)

In [510]:
convert(Array{AbstractString},[string(s) for s in names(train)])

8-element Array{AbstractString,1}:
 "Pclass"       
 "Age"          
 "SibSp"        
 "Parch"        
 "Fare"         
 "EmbarkedLabel"
 "SexLabel"     
 "Survived"     

In [167]:
convert(Array,model)

714x7 Array{Real,2}:
 0  3  22.0  1  0   7.25     true
 1  1  38.0  1  0  71.2833  false
 1  3  26.0  0  0   7.925   false
 1  1  35.0  1  0  53.1     false
 0  3  35.0  0  0   8.05     true
 0  1  54.0  0  0  51.8625   true
 0  3   2.0  3  1  21.075    true
 1  3  27.0  0  2  11.1333  false
 1  2  14.0  1  0  30.0708  false
 1  3   4.0  1  1  16.7     false
 1  1  58.0  0  0  26.55    false
 0  3  20.0  0  0   8.05     true
 0  3  39.0  1  5  31.275    true
 ⋮                  ⋮            
 0  3  19.0  0  0   7.8958   true
 1  1  56.0  0  1  83.1583  false
 1  2  25.0  0  1  26.0     false
 0  3  33.0  0  0   7.8958   true
 0  3  22.0  0  0  10.5167  false
 0  2  28.0  0  0  10.5      true
 0  3  25.0  0  0   7.05     true
 0  3  39.0  0  5  29.125   false
 0  2  27.0  0  0  13.0      true
 1  1  19.0  0  0  30.0     false
 1  1  26.0  0  0  30.0      true
 0  3  32.0  0  0   7.75     true

In [57]:
function scatterMatrix(olddf, colorido=[], legenda=true)
    df = olddf[complete_cases(olddf),:]
    n = size(df, 2)
    nomes = names(df)
    if colorido!=[]
        n = n-1
        splice!(nomes,findfirst(x -> string(x) == colorido, nomes))
    end
    M = Array(Compose.Context, (n,n))

    for (i,indexi) in zip(nomes,1:length(nomes))
        nowcor = false
        for (j,indexj) in zip(nomes,1:length(nomes))
            gdplot = Geom.point
            xTickMarks=yTickMarks=false
            xName=yName=""
            kps=:none
            if j == nomes[1]
                yName=string(i)
                yTickMarks=true
            end
            if i == nomes[end]
                xName=string(j)
                xTickMarks=true
            end
            if nowcor#Cor Info
                index1 = complete_cases(df[:,[i,j]])
                text0 = "Corr: "*string(trunc(cor(df[index1,i],df[index1,j]),4))
                M[indexi,indexj] = compose(context(),
                (context(), text(0.5, 0.5, text0, hcenter)),
                (context(0.1w, 0.1h, 0.8w, 0.8h), rectangle(), fill("white"), stroke("black")))#))
            elseif i == j #histogram
                gdplot = Geom.histogram
                if legenda
                    kps=:right
                end
                M[indexi,indexj] = render(plot(df, x=string(j), y=string(i), color=colorido, gdplot(maxbincount=20),
                Guide.xlabel(xName), Guide.ylabel(yName), Guide.xticks(label=xTickMarks), Guide.yticks(label=yTickMarks), 
                Theme(grid_line_width=1pt, panel_stroke=colorant"black", key_position=kps)))
                nowcor=true
            else #scatterplots
                M[indexi,indexj] = render(plot(df, x=string(j), y=string(i), color=colorido, gdplot,
                Guide.xlabel(xName), Guide.ylabel(yName), Guide.xticks(label=xTickMarks), Guide.yticks(label=yTickMarks), 
                Theme(panel_stroke=colorant"black", key_position=kps)))
            end
        end
    end
    return gridstack(M)
end

scatterMatrix (generic function with 3 methods)